# Pipeline Project

You will be using the provided data to create a machine learning model pipeline.

You must handle the data appropriately in your pipeline to predict whether an
item is recommended by a customer based on their review.
Note the data includes numerical, categorical, and text data.

You should ensure you properly train and evaluate your model.

## The Data

The dataset has been anonymized and cleaned of missing values.

There are 8 features for to use to predict whether a customer recommends or does
not recommend a product.
The `Recommended IND` column gives whether a customer recommends the product
where `1` is recommended and a `0` is not recommended.
This is your model's target/

The features can be summarized as the following:

- **Clothing ID**: Integer Categorical variable that refers to the specific piece being reviewed.
- **Age**: Positive Integer variable of the reviewers age.
- **Title**: String variable for the title of the review.
- **Review Text**: String variable for the review body.
- **Positive Feedback Count**: Positive Integer documenting the number of other customers who found this review positive.
- **Division Name**: Categorical name of the product high level division.
- **Department Name**: Categorical name of the product department name.
- **Class Name**: Categorical name of the product class name.

The target:
- **Recommended IND**: Binary variable stating where the customer recommends the product where 1 is recommended, 0 is not recommended.

## Load Data

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv(
    "data/reviews.csv",
)

df.info()
df.head()

## Preparing features (`X`) & target (`y`)

In [ ]:
data = df

# separate features from labels
X = data.drop("Recommended IND", axis=1)
y = data["Recommended IND"].copy()

print("Labels:", y.unique())
print("Features:")
display(X.head())

In [ ]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.1,
    shuffle=True,
    random_state=27,
)

## Load NLP

In [ ]:
# ! python -m spacy download en_core_web_sm

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Your Work

## Data Exploration

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Numerical Features
num_features = ["Clothing ID", "Age", "Positive Feedback Count"]

fig, axs = plt.subplots(2, 3, figsize=(15, 10))

i = 0
# histogram of numerical features
for feature in num_features:
    ax = axs[i // 3, i % 3]
    data[feature].plot.hist(ax=ax)
    ax.set_title(feature)
    ax.legend()
    i += 1

# box and whisker plot of numerical features split by Recommended IND
for feature in num_features:
    ax = axs[i // 3, i % 3]
    data.boxplot(column=feature, by="Recommended IND", ax=ax, vert=False)
    ax.set_title("")
    ax.set_title(feature)
    ax.tick_params(axis="x", rotation=0)
    i += 1

### Numerical Features

The numerical features are `Clothing ID`, `Age`, and `Positive Feedback Count`.
Comparing the box and whisker plots of the numerical features for both the recommended and not recommended products shows that the distributions are similar so these features may not be useful in predicting the target.

In [ ]:
# Categorical Features
cat_features = ["Division Name", "Department Name", "Class Name"]

fig, axs = plt.subplots(2, 3, figsize=(15, 10))

cat_order_by_feature = {}
i = 0
#  categorical features
for feature in cat_features:
    cat_order_by_feature[feature] = data[feature].value_counts().index
    ax = axs[i // 3, i % 3]
    data[feature].value_counts().reindex(cat_order_by_feature[feature]).plot.bar(
        alpha=0.6, ax=ax
    )
    i += 1

# categorical features as percentage by Recommended IND
for feature in cat_features:
    ax = axs[i // 3, i % 3]
    data.groupby([feature, "Recommended IND"]).size().unstack().apply(
        lambda x: x / x.sum(), axis=1
    ).reindex(cat_order_by_feature[feature]).plot.bar(stacked=True, ax=ax)
    i += 1

### Categorical Features

The categorical features are `Division Name`, `Department Name`, and `Class Name`.
Comparing the value counts of the categorical features for both the recommended and not recommended products shows that the distributions are similar so these features may not be useful in predicting the target.

In [ ]:
# Text Features
text_features = ["Title", "Review Text"]

fig, axs = plt.subplots(2, 2, figsize=(15, 10))

i = 0
# text features
for feature in text_features:
    ax = axs[i // 2, i % 2]
    # count the number of words in each review
    length_feature = f"{feature} word count"
    data[length_feature] = data[feature].str.split().apply(len)
    bin_size = data[length_feature].max() // 10
    bins = range(0, data[length_feature].max() + bin_size, bin_size)

    for recommended in [0, 1]:
        data[data["Recommended IND"] == recommended][length_feature].plot.hist(
            alpha=0.6, ax=ax, label=f"Recommended {recommended}", bins=bins
        )
    i += 1

# Make box and whisker plot of word count split by Recommended IND
for feature in text_features:
    ax = axs[i // 2, i % 2]
    data.boxplot(
        column=f"{feature} word count", by="Recommended IND", ax=ax, vert=False
    )
    i += 1

### Text Features

The text features are `Title` and `Review Text`.
From the box and whisker plots we can see that items that are recommended have slightly shorter title word counts. The review text word counts are similar for both recommended and not recommended items.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=50,
    max_df=0.9,
    min_df=0.1,
    ngram_range=(1, 2),
)

fig, axs = plt.subplots(1, 2, figsize=(16, 8))

for recommended in [0, 1]:
    X_tfidf = tfidf.fit_transform(
        data[data["Recommended IND"] == recommended]["Review Text"]
    )
    tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
    tfidf_df.sum().sort_values().plot.barh(
        ax=axs[recommended], title=f"Recommended {recommended}"
    )

### Term Frequency-Inverse Document Frequency (TF-IDF)

Looking at the TF-IDF for the Review Text we can see that some of the most common words appear similarly ranked in both the recommended and not recommended items (e.g. "dress", "fit"), but other words have different rankings between the two classes.

For Recommended items, the top 5 words that are ranked higher are "love", "size", great", "wear", and "perfect".
For Not Recommended items, the top 5 words that are ranked higher are "fabric", "look", "small", "material", and "ordered".

These words may be useful in predicting the target.

## Building Pipeline

## Training Pipeline

## Fine-Tuning Pipeline